In [1]:
import ultralytics

In [2]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from tracker import Tracker
import time

model = YOLO('yolov8s.pt')

#cap = cv2.VideoCapture('highway.mp4')
cap = cv2.VideoCapture('highway_mini.mp4')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

count = 0
tracker = Tracker()
down = {}
up = {}
counter_down = []
counter_up = []

red_line_y = 198
blue_line_y = 268
offset = 6

# Create a folder to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            list.append([x1, y1, x2, y2])
    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        if red_line_y<(cy+offset) and red_line_y > (cy-offset):
           down[id]=time.time()   # current time when vehichle touch the first line
        if id in down:
          
           if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
             elapsed_time=time.time() - down[id]  # current time when vehicle touch the second line. Also we a re minusing the previous time ( current time of line 1)
             if counter_down.count(id)==0:
                counter_down.append(id)
                distance = 10 # meters - distance between the 2 lines is 10 meters
                a_speed_ms = distance / elapsed_time
                a_speed_kh = a_speed_ms * 3.6*8  # this will give kilometers per hour for each vehicle. This is the condition for going downside
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh))+'Km/h',(x4,y4 ),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

                
        #####going UP#####     
        if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
           up[id]=time.time()
        if id in up:

           if red_line_y<(cy+offset) and red_line_y > (cy-offset):
             elapsed1_time=time.time() - up[id]
             # formula of speed= distance/time  (distance travelled and elapsed time) Elapsed time is It represents the duration between the starting point and the ending point of the movement.
             if counter_up.count(id)==0:
                counter_up.append(id)      
                distance1 = 10 # meters  (Distance between the 2 lines is 10 meters )
                a_speed_ms1 = distance1 / elapsed1_time
                a_speed_kh1 = a_speed_ms1 * 3.6*8
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh1))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)



    
    text_color = (0, 0, 0)  # Black color for text
    yellow_color = (0, 255, 255)  # Yellow color for background
    red_color = (0, 0, 255)  # Red color for lines
    blue_color = (255, 0, 0)  # Blue color for lines

    cv2.rectangle(frame, (0, 0), (250, 90), yellow_color, -1)

    cv2.line(frame, (172, 198), (774, 198), red_color, 2)
    cv2.putText(frame, ('Red Line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, 268), (927, 268), blue_color, 2)
    cv2.putText(frame, ('Blue Line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Save frame
    frame_filename = f'detected_frames/frame_{count}.jpg'
    cv2.imwrite(frame_filename, frame)

    out.write(frame)

    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
    #if cv2.waitKey(0) & 0xFF == 27:
        break
        

cap.release()
out.release()
cv2.destroyAllWindows()



0: 320x640 9 cars, 3 trucks, 722.7ms
Speed: 8.0ms preprocess, 722.7ms inference, 14.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 753.9ms
Speed: 8.0ms preprocess, 753.9ms inference, 4.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 798.3ms
Speed: 5.9ms preprocess, 798.3ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 867.5ms
Speed: 4.0ms preprocess, 867.5ms inference, 4.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 605.8ms
Speed: 3.7ms preprocess, 605.8ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 626.2ms
Speed: 4.9ms preprocess, 626.2ms inference, 5.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 635.6ms
Speed: 5.4ms preprocess, 635.6ms inference, 4.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 509.1ms
Speed: 6.

Speed: 3.0ms preprocess, 401.2ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 5 trucks, 436.5ms
Speed: 4.2ms preprocess, 436.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 4 trucks, 454.7ms
Speed: 2.0ms preprocess, 454.7ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 4 trucks, 487.2ms
Speed: 3.0ms preprocess, 487.2ms inference, 4.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 3 trucks, 498.5ms
Speed: 2.0ms preprocess, 498.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 4 trucks, 474.2ms
Speed: 2.0ms preprocess, 474.2ms inference, 3.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 445.7ms
Speed: 3.9ms preprocess, 445.7ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 410.0ms
Speed: 2.9ms preprocess, 410.0ms inference, 2.9m


0: 320x640 11 cars, 2 trucks, 356.9ms
Speed: 1.9ms preprocess, 356.9ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 3 trucks, 349.4ms
Speed: 2.5ms preprocess, 349.4ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 3 trucks, 356.3ms
Speed: 2.0ms preprocess, 356.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 truck, 366.0ms
Speed: 2.0ms preprocess, 366.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 2 trucks, 331.1ms
Speed: 2.0ms preprocess, 331.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 2 trucks, 350.2ms
Speed: 2.0ms preprocess, 350.2ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 339.4ms
Speed: 3.0ms preprocess, 339.4ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 truck, 345.8ms
Speed


0: 320x640 9 cars, 1 truck, 363.4ms
Speed: 3.0ms preprocess, 363.4ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 truck, 338.3ms
Speed: 3.3ms preprocess, 338.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 truck, 357.0ms
Speed: 2.0ms preprocess, 357.0ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 340.5ms
Speed: 2.0ms preprocess, 340.5ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 truck, 353.2ms
Speed: 3.0ms preprocess, 353.2ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 truck, 368.0ms
Speed: 3.0ms preprocess, 368.0ms inference, 4.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 truck, 352.0ms
Speed: 3.2ms preprocess, 352.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 truck, 346.2ms
Speed: 2.9ms prep